In [17]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import itertools

import pandas as pd
import numpy as np

def fsmodel():
    model = ConcreteModel()
    
    # 5D Array
    model.gen = Set(initialize=[1, 2])  # Generators
    model.Periods = Set(initialize=[1, 2, 3])  # Time periods
    model.days = Set(initialize=[1, 2, 3])  # Demand scenario for time period 1
    
    # Total Scenarios
    
    file_path = 'newgenexp.xlsx'
    sheet_name = 'DIC'  # Specify the sheet name or sheet number
    
    # Load the data into a pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    c = {}
    for i in range(0,df.shape[0]):
        c[df['Generator'][i]] = df['Ammortized Fixed operating cost($)'][i]
    model.c = Param(model.gen, initialize=c)  # Fixed cost per unit capacity of generator j
    
    sheet_name = 'FIXOP'  # Specify the sheet name or sheet number
    
    # Load the data into a pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    f = {}
    for i in range(0,df.shape[0]):
        f[(df['Generator'][i],df['Period'][i])] = df['FIXOP'][i]
    model.f = Param(model.gen,model.Periods, initialize=f)  # Fixed cost per unit capacity of generator j
    
    sheet_name = 'Additional cost'  # Specify the sheet name or sheet number
    
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    g = {}
    for i in range(0,df.shape[0]):
        g[df['Day'][i]] = df['Cost'][i]
    model.g = Param(model.days, initialize=g)
    
    
    
    d = {}
    
    sheet_name = 'Demand'
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    for i in range(0,df.shape[0]):
        d[df['Period'][i],df['Day'][i]]= df['Demand (KWh)'][i]
        
                           
                   
    model.d = Param(model.Periods,model.days, initialize=d)  # Fixed cost per unit capacity of generator j
    
                           
       
    a = {}    
    sheet_name = 'Availability'
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    for i in range(0,df.shape[0]):
        a[df['Period'][i],df['Day'][i],df['Generator'][i]]= df['Availibility'][i]
        
                           
    model.a = Param(model.Periods,model.days,model.gen, initialize=a)  # Fixed cost per unit capacity of generator j
    
    # Variables
    model.x = Var(model.gen, domain=NonNegativeReals)  # Installed capacity of generator j
    model.y = Var(model.Periods, model.days, model.gen, domain=NonNegativeReals)  # Operating level of generator j
    model.y_purchased = Var(model.Periods, model.days, domain=NonNegativeReals)  # Additional capacity purchased
    # Objective function
    def objective_rule(model):
        fixed_cost = 3*sum(model.c[j] * model.x[j] for j in model.gen)
        operating_cost = sum(sum(sum(model.f[j,i] * model.y[i,k, j] for j in model.gen) + model.g[k] * model.y_purchased[i,k] for i in model.Periods) for k in model.days)
        return fixed_cost + operating_cost
    
    model.obj = Objective(rule=objective_rule, sense=minimize)
    
    # Constraints
    #Constraint for fixing x values
    

    # Demand satisfaction constraints
    def demand_satisfaction_rule(model, i, k):
        return sum(model.y[i,k,j] for j in model.gen) + model.y_purchased[i,k] >= model.d[i,k]
    model.demand_satisfaction_constraint = Constraint(model.Periods, model.days, rule=demand_satisfaction_rule)
    
    # Availability constraints
    def availability_rule(model, i, k, gen):
        return model.y[i,k,gen] <= model.a[i,k,gen] * model.x[gen]
    model.availability_constraint = Constraint(model.Periods, model.days, model.gen, rule=availability_rule)
    return model


In [21]:
file_path = 'newgenexp.xlsx'
sheet_name = 'FIXOP'  # Specify the sheet name or sheet number

# Load the data into a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)
f = {}
for i in range(0,df.shape[0]):
    f[(df['Generator'][i],df['Period'][i])] = df['FIXOP'][i]

sheet_name = 'Additional cost'  # Specify the sheet name or sheet number

df = pd.read_excel(file_path, sheet_name=sheet_name)
g = {}
for i in range(0,df.shape[0]):
    g[df['Day'][i]] = df['Cost'][i]

dem = {}

sheet_name = 'Demand'
df = pd.read_excel(file_path, sheet_name=sheet_name)
for i in range(0,df.shape[0]):
    dem[df['Period'][i],df['Day'][i]]= df['Demand (KWh)'][i]
    
av = {}    
sheet_name = 'Availability'
df = pd.read_excel(file_path, sheet_name=sheet_name)
for i in range(0,df.shape[0]):
    av[df['Period'][i],df['Day'][i],df['Generator'][i]]= df['Availibility'][i]

sheet_name = 'DIC'  # Specify the sheet name or sheet number

# Load the data into a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)
c = {}
for i in range(0,df.shape[0]):
    c[df['Generator'][i]] = df['Ammortized Fixed operating cost($)'][i]
days = [1,2,3]
periods = [1,2,3]
gen = [1,2]    
def subproblem_lag(k,nu,mu):
    sp = ConcreteModel()
    sp.gen = Set(initialize=[1, 2])  # Generators
    sp.Periods = Set(initialize=[1, 2, 3])
    sp.x = Var(sp.gen, domain=NonNegativeReals)
    sp.y = Var(sp.Periods,sp.gen, domain=NonNegativeReals)  # Operating level of generator j
    sp.y_purchased = Var(sp.Periods, domain=NonNegativeReals)  # Additional capacity purchased
    if(k==1):
        c_mod = [c[1]+nu[0]+mu[0],c[2]+nu[1]+mu[1]]
    elif(k==2):
        c_mod = [-nu[0]+c[1],-nu[1]+c[2]]
    else:
        c_mod = [-mu[0]+c[1],-mu[1]+c[2]]
    def objective_rule(model):
        fixed_cost = 1*sum(c_mod[j-1] * model.x[j] for j in model.gen)
        operating_cost = sum(sum(f[j,i] * model.y[i,j] for j in model.gen) + g[k] * model.y_purchased[i] for i in model.Periods)
        return operating_cost+fixed_cost
    
    sp.obj = Objective(rule=objective_rule, sense=minimize)
    
    # Constraints
    #Constraint for fixing x values
    def up_bound(model, j):
        return model.x[j] <=10000 
    sp.up_bound_constraint = Constraint(sp.gen, rule=up_bound)

    # Demand satisfaction constraints
    def demand_satisfaction_rule(model, i):
        return -sum(model.y[i,j] for j in model.gen) - model.y_purchased[i] <=-dem[i,k] 
    sp.demand_satisfaction_constraint = Constraint(sp.Periods, rule=demand_satisfaction_rule)
    
    # Availability constraints
    def availability_rule(model, i, gen):
        return model.y[i,gen]- av[i,k,gen] * model.x[gen] <= 0
    sp.availability_constraint = Constraint(sp.Periods, sp.gen, rule=availability_rule)
    return sp
    
    

In [22]:
def heuristic(xval):
    if(xval==10**20):
        return 10**20
    model = fsmodel()
    def cap_fix_rule(model, gen):
        return model.x[gen] == xval[gen]
    model.cap_fix_constraint = Constraint(model.gen, rule=cap_fix_rule)
    opt = SolverFactory('gurobi')
    results = opt.solve(model, tee=False)
    
    
    #model.display()
    if results.solver.termination_condition == TerminationCondition.infeasible:
        return 10**20
    return value(model.obj)

In [31]:
UB = float('inf')
LB = float('-inf')
nu = np.array([100,100])
mu = np.array([100,100])
tol = 10**(-6)
Best_sol = [0,0]
opt = SolverFactory('gurobi',tee=False)
while(UB-LB>tol):
    z_ld = 0
    xval_full = {}
    for k in days:
        sp = subproblem_lag(k,nu,mu)
        opt.solve(sp)
        z_ld = z_ld+sp.obj()
        xval = {1:sp.x[1](),2:sp.x[2]()}
        xval_full[k,1] = sp.x[1]()
        xval_full[k,2] = sp.x[2]()
        if(heuristic(xval)<UB):
            UB = heuristic(xval)
            Best_sol = xval
    LB = max(LB,z_ld)
    if(UB-LB)<=tol*UB:
        break
    tk = (UB-z_ld)/((xval_full[2,1]-xval_full[1,1])**2+(xval_full[3,1]-xval_full[1,1])**2+(xval_full[2,2]-xval_full[1,2])**2+(xval_full[3,2]-xval_full[1,2])**2)
    x_diff = np.array([tk*(xval_full[1,1] - xval_full[2,1]),tk*(xval_full[1,2] - xval_full[2,2]),tk*(xval_full[1,1]-xval_full[3,1]),tk*(xval_full[1,2]-xval_full[3,2])])
    print(UB)
    print(LB)
    print((UB-LB)/UB)
    print(tk)
    print(Best_sol)
    nu[0] = nu[0]+x_diff[0]
    nu[1] = nu[1]+x_diff[1]
    mu[0] = mu[0]+x_diff[2]
    mu[1] = mu[1]+x_diff[3]

    

504999.525
-3062333.3600000003
7.064032159238169
0.008918332212500001
{1: 0.0, 2: 0.0}
363166.54000000004
265560.4793939394
0.26876391367459307
0.0076722920747710815
{1: 3500.0, 2: 0.0}
363166.54000000004
265560.4793939394
0.26876391367459307
0.005446433130201779
{1: 3500.0, 2: 0.0}
363166.54000000004
298408.96424242423
0.17831371733083062
0.0027462250649843765
{1: 3500.0, 2: 0.0}
363166.54000000004
307929.2662626263
0.15209901698921316
0.0026520450476885636
{1: 3500.0, 2: 0.0}
363166.54000000004
310676.641010101
0.14453396226948392
0.002474080521888216
{1: 3500.0, 2: 0.0}
359290.31371428573
316475.5595613275
0.1191647882469921
0.008762299622677511
{1: 1876.2082857142855, 2: 1619.0285714285715}
359290.31371428573
316475.5595613275
0.1191647882469921
0.000708932478631447
{1: 1876.2082857142855, 2: 1619.0285714285715}
359290.31371428573
325105.9348282828
0.09514417055280533
0.001409363912376976
{1: 1876.2082857142855, 2: 1619.0285714285715}
359290.31371428573
332267.5536565656
0.07521149

KeyboardInterrupt: 